In [6]:
from garmin_fit_sdk import Decoder, Stream

stream = Stream.from_file("20251001.fit")
decoder = Decoder(stream)
messages, errors = decoder.read()


In [ ]:
# print(errors)
print(messages)

In [18]:
messages.keys()

dict_keys(['file_id_mesgs', 'software_mesgs', 'record_mesgs', 'session_mesgs', 'activity_mesgs'])

In [20]:
messages['file_id_mesgs']

[{'serial_number': 224002150,
  'time_created': datetime.datetime(2025, 10, 1, 4, 22, 17, tzinfo=datetime.timezone.utc),
  'product_name': '2851-229',
  'manufacturer': 'magene',
  'product': 96,
  'type': 'activity'}]

In [24]:
messages['session_mesgs']

[{'timestamp': datetime.datetime(2025, 10, 1, 12, 27, 13, tzinfo=datetime.timezone.utc),
  'start_time': datetime.datetime(2025, 10, 1, 4, 22, 17, tzinfo=datetime.timezone.utc),
  'sport': 'cycling',
  'sub_sport': 'road',
  'max_temperature': 32,
  'avg_temperature': 25,
  'total_ascent': 114,
  'total_distance': 150072.61,
  'total_elapsed_time': 29096.0,
  'total_timer_time': 21898.0,
  'total_moving_time': 21898.0,
  'total_calories': 4815,
  'total_work': 0,
  'max_power': 0,
  'max_speed': 10.75,
  'max_cadence': 0,
  'max_heart_rate': 0,
  'avg_power': 0,
  'avg_speed': 6.852,
  'avg_cadence': 0,
  'avg_heart_rate': 0,
  'enhanced_avg_speed': 6.852,
  'enhanced_max_speed': 10.75}]

In [28]:
21898.0 / 60 / 60

6.082777777777777

In [34]:
10.75 * 3.6

38.7

In [32]:
38 / 10.75

3.5348837209302326

In [36]:
6.852 * 3.6

24.6672

In [38]:
messages['activity_mesgs']

[{'timestamp': datetime.datetime(2025, 10, 1, 12, 27, 13, tzinfo=datetime.timezone.utc),
  'total_timer_time': 22080.0,
  'local_timestamp': 1128284836,
  'num_sessions': 1,
  'type': 'auto_multi_sport',
  'event': 1,
  'event_type': 'stop',
  'event_group': 1}]

In [42]:
records = messages['record_mesgs']

In [78]:
record_keys = set()

In [ ]:
[[record_keys.add(key) for key in record.keys()] for record in records]

In [82]:
record_keys

{'altitude',
 'distance',
 'enhanced_altitude',
 'enhanced_speed',
 'grade',
 'position_lat',
 'position_long',
 'speed',
 'temperature',
 'timestamp'}

In [50]:
record_keys.add(2)

In [56]:
record_keys

{1, 2}

In [76]:
for i in records:
    if ('1' in i or '2' in i):
        print(i)

In [146]:
demo_record = records[::20][:10]

In [114]:
data_time = demo_record[0]['timestamp']

In [138]:
data_time.timestamp()

1759292619.0

In [134]:
data_time = data_time.astimezone()

In [136]:
data_time

datetime.datetime(2025, 10, 1, 12, 23, 39, tzinfo=datetime.timezone(datetime.timedelta(seconds=28800), 'CST'))

In [ ]:
demo_record

In [160]:
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple

def plot_compact_trajectory(compact_coords: List[Tuple[int, int]], 
                           title="GPS轨迹图", 
                           show_stats=True):
    """
    绘制紧凑格式坐标的轨迹图
    
    Args:
        compact_coords: 紧凑格式坐标列表 [(lat1, lon1), (lat2, lon2), ...]
        title: 图表标题
        show_stats: 是否显示统计信息
    """
    
    # 转换为标准坐标
    standard_coords = []
    lats, lons = [], []
    
    for compact_lat, compact_lon in compact_coords:
        lat_deg = compact_lat * (180.0 / 2**31)
        lon_deg = compact_lon * (180.0 / 2**31)
        standard_coords.append((lat_deg, lon_deg))
        lats.append(lat_deg)
        lons.append(lon_deg)
    
    # 创建图表
    plt.figure(figsize=(12, 8))
    
    # 绘制轨迹线
    plt.plot(lons, lats, 'b-', alpha=0.7, linewidth=2, label='运动轨迹')
    
    # 标记起点和终点
    plt.plot(lons[0], lats[0], 'go', markersize=10, label='起点')
    plt.plot(lons[-1], lats[-1], 'ro', markersize=10, label='终点')
    
    # 添加一些中间点标记
    if len(lons) > 10:
        step = len(lons) // 5
        for i in range(1, 5):
            idx = i * step
            plt.plot(lons[idx], lats[idx], 'yo', markersize=6)
    
    plt.xlabel('经度')
    plt.ylabel('纬度')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 显示统计信息
    if show_stats:
        total_distance = calculate_total_distance(standard_coords)
        plt.figtext(0.02, 0.02, 
                   f"点数: {len(compact_coords)} | "
                   f"总距离: {total_distance:.2f} km | "
                   f"内存节省: {len(compact_coords) * 8 / 1024:.1f} KB", 
                   fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # return standard_coords

def calculate_total_distance(coords):
    """计算轨迹总距离（简化版）"""
    total_km = 0
    for i in range(1, len(coords)):
        # 使用简化的距离计算（实际应用应使用Haversine公式）
        lat1, lon1 = coords[i-1]
        lat2, lon2 = coords[i]
        distance = np.sqrt((lat2-lat1)**2 + (lon2-lon1)**2) * 111
        total_km += distance
    return total_km

In [156]:
positions = [(record['position_lat'], record['position_long']) for record in records if 'position_lat' in record]

In [ ]:
plot_compact_trajectory(positions)

In [1]:
from parser import Fitparser

ModuleNotFoundError: No module named 'parser'

In [5]:
from typing import List

In [7]:
res_list: List[str] = ['abc', 'def']

In [9]:
res_list[1]

'def'